In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Butaritari,KI,2022-05-18 17:15:21,3.0707,172.7902,80.92,77,52,8.75
1,1,Vila Franca Do Campo,PT,2022-05-18 17:21:06,37.7167,-25.4333,64.20,92,100,14.50
2,2,Port Alfred,ZA,2022-05-18 17:21:07,-33.5906,26.8910,73.18,45,31,1.99
3,3,Cape Town,ZA,2022-05-18 17:21:07,-33.9258,18.4232,61.21,90,40,10.36
4,4,Hobart,AU,2022-05-18 17:13:07,-42.8794,147.3294,50.25,69,31,5.75


In [4]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [5]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)


In [18]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [19]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [22]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Butaritari,KI,2022-05-18 17:15:21,3.0707,172.7902,80.92,77,52,8.75
5,5,Kapaa,US,2022-05-18 17:16:56,22.0752,-159.3190,75.18,93,100,17.27
6,6,Pitimbu,BR,2022-05-18 17:21:08,-7.4706,-34.8086,86.59,65,42,8.19
7,7,Birpur,IN,2022-05-18 17:21:08,26.5167,87.0167,81.77,63,20,8.48
9,9,Sao Filipe,CV,2022-05-18 17:18:11,14.8961,-24.4956,78.94,61,95,8.28
13,13,Rikitea,PF,2022-05-18 17:15:22,-23.1203,-134.9692,77.38,69,85,15.82
14,14,Batticaloa,LK,2022-05-18 17:21:11,7.7102,81.6924,84.02,67,56,3.71
15,15,Hirara,JP,2022-05-18 17:21:11,24.8000,125.2833,75.15,76,71,14.43
17,17,Gat,IL,2022-05-18 17:19:29,31.6100,34.7642,75.40,7,0,8.57
18,18,Salalah,OM,2022-05-18 17:16:40,17.0151,54.0924,84.29,84,75,4.61


In [31]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Butaritari,KI,80.92,3.0707,172.7902,
5,Kapaa,US,75.18,22.0752,-159.3190,
6,Pitimbu,BR,86.59,-7.4706,-34.8086,
7,Birpur,IN,81.77,26.5167,87.0167,
9,Sao Filipe,CV,78.94,14.8961,-24.4956,
13,Rikitea,PF,77.38,-23.1203,-134.9692,
14,Batticaloa,LK,84.02,7.7102,81.6924,
15,Hirara,JP,75.15,24.8000,125.2833,
17,Gat,IL,75.40,31.6100,34.7642,
18,Salalah,OM,84.29,17.0151,54.0924,


In [32]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [41]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [42]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Butaritari,KI,80.92,3.0707,172.7902,Isles Sunset Lodge
5,Kapaa,US,75.18,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
6,Pitimbu,BR,86.59,-7.4706,-34.8086,Reserva do Abiaí
7,Birpur,IN,81.77,26.5167,87.0167,Maa Sherawali Sweet Content
9,Sao Filipe,CV,78.94,14.8961,-24.4956,Hotel Xaguate
13,Rikitea,PF,77.38,-23.1203,-134.9692,People ThankYou
14,Batticaloa,LK,84.02,7.7102,81.6924,Grand Star Guest House
15,Hirara,JP,75.15,24.8000,125.2833,Hotel Atoll Emerald Miyakojima
17,Gat,IL,75.40,31.6100,34.7642,OlusHome
18,Salalah,OM,84.29,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"


In [43]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [44]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [47]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [48]:
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [49]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))